# <b> Canal de diffusion anonyme et génération de secret </b>

<i> Compte-rendu du TP2 par Valentin SÉRÉE et Jérémy SERENA [INFO3]</i>

In [1]:
from datetime import datetime

class Forum:
    messages = []

    def init(self):
        self.messages = []

    def recupDate(self):
        dateBrute = str(datetime.now())
        date = dateBrute[11:13] + "h" + dateBrute[14:16] + "m" + dateBrute[17:19] + "s"
        return date

    def calculDate(self, date2):
            msg_date = list(date2)
            date = ""
            i=0
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            heure = int(date[0:2])
            date = ""
            i += 1
            while msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            minute = int(date[0:2])
            date = ""
            i += 1
            while len(msg_date) and msg_date[i].isdigit() :
                date += msg_date[i]
                i+=1
            seconde = int(date[0:2])
            total = 3600*heure+60*minute+seconde
            return (total)
    
    def recupererMessagesAnonymes(self, date_debut, date_fin) :
        for msg in self.messages :
            date = self.calculDate(msg[1])
            date_start = self.calculDate(date_debut)
            date_end = self.calculDate(date_fin)
            if date > date_start and date < date_end :
                print(msg[0])

    def posterMessageAnonyme(self, message) :
        date = self.recupDate()
        self.messages.append([message, date])



In [2]:
if __name__ == "__main__" :
    print("start")
    forum = Forum()
    while(True) :
        string = input()
        if "show_message" == string :
            print("entrez la date au format 11h02m03s" )
            date_debut = input()
            date_fin = input()
            forum.recupererMessagesAnonymes(date_debut, date_fin)
        else :
            forum.posterMessageAnonyme(string)

start


KeyboardInterrupt: Interrupted by user

In [7]:
import time
import random
import threading

class Bob() :

    def __init__(self) :
        self.name = "Bob"
        self.partner = "Alice"
        self.duree = 20

    def genererSecret(self) :
        attente = random.randint(0, 2)
        bit = random.randint(0, 1)
        if bit == 0 :
            message = self.name
        else :
            message = self.partner
        self.forum.posterMessageAnonyme(message)
        self.duree -= attente

class Alice() :

    def __init__(self) :
        self.name = "Alice"
        self.partner = "Bob"
        self.duree = 20

    def genererSecret(self) :
        attente = random.randint(0, 2)
        bit = random.randint(0, 1)
        if bit == 0 :
            message = self.name
        else :
            message = self.partner
        self.forum.posterMessageAnonyme(message)
        self.duree -= attente


def exec1() :
    threading.Thread(bob.genererSecret()).start()

def exec2() :
    threading.Thread(alice.genererSecret()).start()

if __name__ == "__main__" :
    print("start")
    forum = Forum()
    bob = Bob()
    alice = Alice()
    bob.forum = forum
    alice.forum = forum
    for i in range(10) :
        delay1 = random.randint(0, 2)
        delay2 = random.randint(0, 2)
        t = threading.Timer(delay1, exec1)
        u = threading.Timer(delay2, exec2)
        t.start()
        u.start()
    time.sleep(5)
    t.join
    u.join()
    print("stop")
    forum.recupererMessagesAnonymes("08l08l08l", "18h10h10h") 

start
stop
['bob', '10h20m30s']
['alice', '10h20m31s']
['alice', '10h20m32s']
['alice', '10h20m32s']
['alice', '10h20m34s']
['alice', '10h20m34s']
['bob', '10h20m36s']
['alice', '10h20m37s']
['bob', '10h20m37s']
['alice', '10h20m39s']
['alice', '10h20m39s']
['alice', '10h20m40s']
['bob', '10h20m41s']
['bob', '10h20m41s']
['bob', '10h20m43s']
['alice', '10h20m45s']
['alice', '10h20m47s']
['alice', '10h20m49s']
['alice', '10h20m49s']
['bob', '10h20m51s']
['bob', '10h20m52s']
['alice', '10h20m53s']
['alice', '10h20m54s']
['bob', '10h20m55s']
['bob', '10h20m57s']
['bob', '10h20m57s']
['bob', '10h20m57s']
['alice', '10h20m58s']
['bob', '10h20m59s']
['bob', '10h21m01s']
['alice', '10h21m02s']
['alice', '10h21m04s']
['alice', '10h21m05s']
['bob', '10h21m06s']
['bob', '10h21m07s']
['alice', '10h21m09s']
['bob', '10h21m11s']
['bob', '10h21m41s']
['alice', '10h21m41s']
['bob', '10h21m42s']
['alice', '10h21m43s']
['bob', '10h21m43s']
['alice', '10h21m43s']
['alice', '10h21m44s']
['alice', '10h21m